In [129]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import keras as kr
import random 
import preprocessor
import operator
import lightgbm as lgb
import statsmodels.api as sm
from preprocessor import prepare_data, get_features, get_upper_bounds, get_lower_bounds
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, LassoCV, Lasso
from sklearn import tree
from sklearn import metrics
from sklearn import preprocessing
import keras.callbacks as cb
from lightgbm import LGBMRegressor
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.optimizers import Adam

# Load in pre-split data
X_train, X_test, y_train, y_test = prepare_data()
features = get_features()

Train data(range):
2019-04-11 21:49:48
2019-06-18 23:27:30
Test data(range):
2019-06-18 23:27:49
2019-08-22 19:05:30


In [130]:
X_train_df = pd.DataFrame(X_train)
X_train_df.columns = features
corr = X_train_df[features].corr()
corr = pd.melt(corr.reset_index(), id_vars='index')
corr.columns = ['x', 'y', 'value']

In [52]:
duplicate = []
for i in range(len(corr)):
    if corr.iloc[i]['x'] == corr.iloc[i]['y']:
        duplicate.append(i)

In [53]:
corr = corr.drop(duplicate, axis=0)
corr = corr.dropna()
corr = corr.reset_index(drop=True)

In [54]:
inds = list(corr.value.abs().argsort()[::-1])

In [61]:
corr.iloc[inds][0:50]

,x,y,value
1003,origin_MADRID,destination_MADRID,-1.000000
2796,fare_Individual Sleeper-Flexible,train_class_Cama G. Clase,1.000000
2579,train_class_Cama G. Clase,fare_Individual Sleeper-Flexible,1.000000
732,destination_MADRID,origin_MADRID,-1.000000
2462,train_type_REGIONAL,fare_Adulto Ida,0.884690
2353,fare_Adulto Ida,train_type_REGIONAL,0.884690
717,destinationpop,origin_MADRID,-0.797077
992,destinationpop,destination_MADRID,0.797077
122,origin_MADRID,destinationpop,-0.797077
127,destination_MADRID,destinationpop,0.797077


In [62]:
drop = ['train_class_Cama G. Clase', 'destination_MADRID', 'fare_Adulto Ida']

In [131]:
# read in data and drop unnamed, start/end date columns
renfedata = pd.read_csv("input/cleaned_data.csv")
# keeping sample data in here for easy processing/debugging --> to remove later
renfedata = renfedata.sample(n=50000)
renfedata = renfedata.drop(columns=['Unnamed: 0','start_date', 'end_date', 'insert_date', 'train_class_Cama G. Clase','fare_Individual Sleeper-Flexible', 'destination_MADRID', 'origin_MADRID', 'fare_Adulto Ida'], axis=1)
features = renfedata.drop(columns=['price'], axis=1)
response = renfedata[['price']]
X_train, X_test, y_train, y_test = train_test_split(features, response, train_size=0.75)
minMax_scaler = preprocessing.MinMaxScaler().fit(X_train)
X_train_scaled = minMax_scaler.transform(X_train)
X_test_scaled = minMax_scaler.transform(X_test)

In [134]:
model = GradientBoostingRegressor(n_estimators=500, max_depth=8, loss='ls', min_samples_split=2, learning_rate=0.01)

In [135]:
y_train = list(np.array(y_train).flatten())
y_test = list(np.array(y_test).flatten())

In [136]:
model.fit(X_train_scaled, y_train)
model.score(X_test_scaled, y_test)

0.9374538812734796

In [137]:
mean_squared_error(y_test, model.predict(X_test_scaled))

40.9509677289871

In [139]:
# # function to compute upper bound for response variable
# def get_upper_bounds(y, percent):
#     float_percent = percent/100
#     upper_bound = y + (y * float_percent)
#     return upper_bound


# # function to compute lower bound for response variable
# def get_lower_bounds(y, percent):
#     float_percent = percent/100
#     lower_bound = y - (y * float_percent)
#     return lower_bound

def get_bounds(y, percent):
    float_percent = percent/100
    upper_bound = y + (y * float_percent)
    lower_bound = y - (y * float_percent)
    lower_bound.columns = ['lower_bound']
    upper_bound.columns = ['upper_bound']
    bounds = lower_bound.join(upper_bound)
    return bounds

# function to compute accuracy scores for predictions based on actual upper/lower bound limits
def get_interval_accuracy_score(bounds, y):
    preds_acc = []
    for i in range(len(y)):
        if (bounds.iloc[i].lower_bound <= y[i] <= bounds.iloc[i].upper_bound):
            preds_acc.append(1)
        else:
            preds_acc.append(0)

    return np.mean(preds_acc)

In [133]:
bounds = get_bounds(y_test, 5)

In [140]:
get_interval_accuracy_score(bounds, model.predict(X_test_scaled))

0.52016